In [6]:
from fastai.text.all import *
from pathlib import Path

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

In [8]:
DATA_ROOT = Path('../data')

df_train = pd.read_csv(DATA_ROOT/'train21_shuffled.csv')
df_test = pd.read_csv(DATA_ROOT/'test21_shuffled.csv')

,season,episode,scene,line_text,speaker,deleted
0,3,16,29,"Don't hurt that bat, Creed! It's a living thing with feelings and a family!",Kelly,False
1,7,7,27,"I cancelled my plans to come to this thing, and they repay me with this?",Kevin,False
2,8,13,25,"Oh, yes. Oh, what a beautiful child. Prominent forehead, short arms, tiny nose. You will lead millions... [whispers] willingly, or as slaves.",Dwight,False
3,2,17,18,Brad Pitt. Also there will be no bonuses.,Dwight,False
4,5,4,32,"Okay, alright. Hey, you know what? I would appreciate it if people would stop storming off the stage.",Michael,False
...,...,...,...,...,...,...
7827,9,7,32,Yeah.,Pam,False
7828,4,3,25,Alright. Well fight it out amongst yourselves. I was thinking Pammy but boys night out is also good.,Michael,False
7829,7,8,19,Rachel.,Kelly,False
7830,8,21,19,[chuckling] Okay.,Andy,False


In [9]:
from transformers import BertForSequenceClassification, BertTokenizer, BertConfig

bert_classes = (BertForSequenceClassification, BertTokenizer, BertConfig)

(transformers.models.bert.modeling_bert.BertForSequenceClassification,
 transformers.models.bert.tokenization_bert.BertTokenizer,
 transformers.models.bert.configuration_bert.BertConfig)

In [10]:
model, tokenizer, config = bert_classes